In [2]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import datetime

### Defined the directory and model parameter

In [94]:
log_directory = "strategy_log/"
file_directory = "data/"

In [ ]:
look_back = 3
portfolio_size = 10

### Extract the data from Yahoo Finance

In [95]:
with open(file_directory+"yahoo_finance_crypto_list.txt") as f:
    
    ticker_list = f.readlines()
    
    ticker_list = [ticker.strip() for ticker in ticker_list]

In [15]:
start_date = (datetime.datetime.today()-datetime.timedelta(days=look_back)).strftime("%Y-%m-%d")
end_date = (datetime.datetime.today()).strftime("%Y-%m-%d")

In [24]:
price_dic = {}
error_list = []

for ticker in ticker_list:
    
    try:
        price_df = get_data(ticker=ticker,
                        start_date=start_date,
                        end_date=end_date,
                        index_as_date=True,
                        interval="1d")
        
        price_dic[ticker] = price_df
    
    except:
        
        print("Error.")
        error_list.append(ticker)
    
    #time.sleep(1)

BTC-USD
ETH-USD
USDT-USD
LTC-USD
XRP-USD
DOT1-USD
DOT2-USD
BCH-USD
ADA-USD
BNB-USD
LINK-USD
USDC-USD
BSV-USD
XLM-USD
EOS-USD
XMR-USD
THETA-USD
TRX-USD
XEM-USD
DOGE-USD
CCXX-USD
XTZ-USD
VET-USD
CEL-USD
ATOM1-USD
ATOM2-USD
NEO-USD
DASH-USD
MIOTA-USD
ZIL-USD
ETC-USD
DFI-USD
ZEC-USD
KSM-USD
DCR-USD
WAVES-USD
ALGO-USD
DGB-USD
ONT-USD
EGLD-USD
OMG-USD
STX1-USD
CTC2-USD
CTC1-USD
CTC3-USD
STX2-USD
LUNA1-USD
LUNA2-USD
BAT-USD
ZRX-USD
AVAX-USD
AIB-USD
ICX-USD
SC-USD
CNX-USD
QTUM-USD
LRC-USD
HBAR-USD
EWT-USD
ABBC-USD
REP-USD
CELO-USD
LRG-USD
TFUEL-USD
XVG-USD
KNC-USD
BTG-USD
MWC-USD
MAID-USD
LSK-USD
NANO-USD
ZEN-USD
NYE-USD
BNT-USD
ANT-USD
DMCH-USD
VITAE-USD
SNT-USD
GRN-USD
RVN-USD
GNO-USD
JUL-USD
SOL1-USD
MONA-USD
BCD-USD
SOL2-USD
MTC1-USD
CKB-USD
KIN-USD
IOST-USD
AR-USD
MTC2-USD
HNT2-USD
XNC-USD
HNT1-USD
AXEL-USD
BTM-USD
BTS-USD
MTC3-USD
ARDR-USD
HNC-USD
BDX-USD
DIVI-USD
GNT-USD
STORJ-USD
BRC-USD
TOMO-USD
VLX-USD
KRT-USD
STEEM-USD
BONO-USD
RLC-USD
KMD-USD
CVC-USD
XDC-USD
SRM-USD
XWC-USD
LBC-USD

In [27]:
price_df_list = [price_dic[ticker] for ticker in ticker_list if ticker not in error_list]
price_df = pd.concat(price_df_list,axis=0)

### Filter the universe by average transaction volume

In [34]:
average_volume = price_df[["volume","ticker"]].groupby("ticker").mean()
volume_threshold = average_volume.describe().loc["25%",:][0]
universe_list = list(average_volume[average_volume>volume_threshold].index)

In [48]:
price_df_list = [price_dic[ticker] for ticker in universe_list]
price_df = pd.concat(price_df_list,axis=0)

### Calculate Signal

In [61]:
price_df["date"] = price_df.index
price_wide = price_df.pivot_table(values="close",columns="ticker",index="date")


signal = price_wide.pct_change(look_back)
signal = signal.iloc[-1,:].dropna()

### Create Portfolios

In [ ]:
top_portfolio = signal.nlargest(portfolio_size)
bottom_portfolio = signal.nsmallest(portfolio_size)

top_portfolio = pd.DataFrame(top_portfolio).T
bottom_portfolio = pd.DataFrame(bottom_portfolio).T

In [ ]:
### Write the Log

In [93]:
top_portfolio.to_csv(log_directory+"top_portfolio"+end_date+".csv")
bottom_portfolio.to_csv(log_directory+"bottom_portfolio"+end_date+".csv")